# Exploring Spatial Filters, as 3D height maps
stough 202-

In [spatial_ops](./spatial_ops.ipynb) we saw how we could `correlate` or filter an image with a particular mask or filter, in order to compute something about the local neighborhood around every pixel in an image. We saw a blurring filter, vertical and horizontal edge detecting filters, and the LaPlacian. Here we'll take a closer look at these filters.

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np

# For spatial filtering/operations
from scipy.ndimage import (correlate,
                           convolve)
from scipy.stats import (norm, 
                         entropy)

# For importing from alternative directory sources
import sys  
sys.path.insert(0, '../dip_utils')

from matrix_utils import (arr_info,
                          make_linmap)
from vis_utils import (vis_rgb_cube,
                       vis_hists,
                       vis_pair)

## Blurring
Though the filters themselves might be small (3x3, 11x11) relative to the size of the image, we're going to upsample them here, so we can look at the ideal versions as 3D heightmaps. 

When blurring, we can use either a flat filter, giving equal weight to every pixel in the neighborhood, or we can do a smoother Gaussian kind of blur, where we give more weight to pixels closer to the center.

In [ ]:
x = np.arange(-2.5,2.5,.1)
y_flat = .2*np.ones_like(x) # times .2 just for scaling of the plot
y_gaussian = norm.pdf(x)

In [ ]:
y_gaussian.sum(), y_flat.sum()

In [ ]:
plt.figure(figsize=(4,3))
plt.plot(x, np.transpose([y_flat, y_gaussian]));

### View in 3D

In [ ]:
X2d, Y2d = np.meshgrid(x, x, indexing='ij')

fig = plt.figure(figsize=(4,4))
ax = fig.add_subplot(111, projection='3d')

ax.plot_surface(X2d, Y2d, np.outer(y_gaussian, y_gaussian), cmap=cm.coolwarm);
plt.tight_layout()

## Edge Detecting Filters
We saw the simple Sobel filters, like:
\begin{equation*}
\mathbf{h_{Sobel}} =  \begin{vmatrix}
1 & 0 & -1 \\
2 & 0 & -2 \\
1 & 0 & -1 
\end{vmatrix}
\end{equation*}

This is really just an approximation of the derivative of a Gaussian in one direction. We can plot this in 3D in ideal form by taking the local difference in one of the dimensions.

In [ ]:
plt.figure(figsize=(4,3))
plt.plot(x, np.transpose([y_gaussian, 10*np.gradient(y_gaussian)])); # times 10 just for scaling.

In [ ]:
fig = plt.figure(figsize=(4,4))
ax = fig.add_subplot(111, projection='3d')

ax.plot_surface(X2d, Y2d, np.outer(np.gradient(y_gaussian), y_gaussian), cmap=cm.coolwarm);
plt.tight_layout()

## Cross-derivative
Whereas the above is a uni-direction edge detecting filter, we also saw the laplacian filter:
\begin{equation*}
\mathbf{h_{Laplace}} =  \begin{vmatrix}
-1 & -1 & -1 \\
-1 & 8 & -1 \\
-1 & -1 & -1 
\end{vmatrix}
\end{equation*}

Turns out, this is just a 3x3 version of the second derivative. We'll negate it just so the peak is high instead of low.

In [ ]:
plt.figure(figsize=(4,3))
plt.plot(x, np.transpose([np.gradient(y_gaussian), 
                          10*np.gradient(np.gradient(y_gaussian))])); # times 10 for scaling

In [ ]:
fig = plt.figure(figsize=(4,4))
ax = fig.add_subplot(111, projection='3d')

ax.plot_surface(X2d, Y2d, np.outer(-np.gradient(np.gradient(y_gaussian)), 
                                   y_gaussian), 
                cmap=cm.coolwarm);
plt.tight_layout()

And the second derivative in both directions:

In [ ]:
fig = plt.figure(figsize=(4,4))
ax = fig.add_subplot(111, projection='3d')

ax.plot_surface(X2d, Y2d, np.outer(np.gradient(np.gradient(y_gaussian)), 
                                   np.gradient(np.gradient(y_gaussian))), 
                cmap=cm.coolwarm);
plt.tight_layout()